# C3S examples 
## Retrieve and visualize NetCDF files

## 1 - Configuration

### 1.1 - SET YOUR C3S ACCOUNT (FOR THE AUTHENTIFICATION) 

In [ ]:
# For Registration (C3S)
url_registration= 'https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome'
from IPython.core.display import display, HTML
display(HTML(f"""<a href={url_registration}>{'Registration_C3S - please click on it'}</a>"""))

### 1.2 - Log in and retrieve your key

In [ ]:
url_login= 'https://cds.climate.copernicus.eu/user/login?destination=%2F%23!%2Fhome'
url_key = 'https://cds.climate.copernicus.eu/api-how-to'

display(HTML(f"""<a href={url_login}>{'Please login - please click on it'}</a>"""))
display(HTML(f"""<a href={url_key}>{'Retrieve your key - please click on it'}</a>"""))

### 1.3- Install the CDS API key

In [ ]:
import os

print('Please enter your key:')
key = input()

content = "url: https://cds.climate.copernicus.eu/api/v2\nkey: " + key

#Write content in $HOME/.cdsapirc 
f = open(os.environ["HOME"] + '/cdsapirc', 'w')
f.write(content)
f.close()
!mv $HOME/cdsapirc $HOME/.cdsapirc

## 2 - Consult the catalogue

#### Choose a dataset, click on "Download data", select the desired parameters, agree to the Terms of Use and finally, click on "Show API request" to see the Python script

In [ ]:
url_dataset= 'https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset'
display(HTML(f"""<a href={url_dataset}>{'Consult the catalogue - please click on it'}</a>"""))

## 3 - Climate projections

### 3.1 - Create a data folder

In [ ]:
#Get the destination folder path
folder = '/home/jovyan/work/c3s_data/c3s_NetCDF_examples/04_Climate_projections'

import os 

#Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

filename = folder + '/2m_temperature'

### 3.2 - Download a file
#### TO DO : Check you agree with the __[Terms of Use](https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip5-monthly-single-levels?tab=form)__ (Last section)
#### Note : Please, go to https://cds.climate.copernicus.eu/cdsapp#!/yourrequests to track the progress of your request

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'projections-cmip5-monthly-single-levels',
    {
        'ensemble_member':'r1i1p1',
        'format':'zip',
        'experiment':'rcp_8_5',
        'period':'200512-203011',
        'model':'hadgem2_cc',
        'variable':'2m_temperature'
    },
    filename)

### 3.3 - Read a file
#### Note : The keys and variables may differ depending on the selected file.

In [ ]:
import netCDF4
from netCDF4 import Dataset
nc =  Dataset(filename, 'r')

In [ ]:
print('Keys :  ' + str(nc.variables.keys()) + '\n')

print("Content of 'time' variable :\n" + str(nc.variables['time']))

### 3.4 - Create a chart

In [ ]:
import calendar
import pandas as pd 
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt 

times = nc.variables['time'][:]
lons = nc.variables['lon'][:]
lats = nc.variables['lat'][:]
values = nc.variables['tas'][:]

start_time = datetime.strptime('1859-12-01', '%Y-%m-%d') #See unit in time variable
 
df = pd.DataFrame(index = [calendar.month_abbr[i] for i in range(1,13)])
 
for i in range(len(times)):
    dt_time = start_time + timedelta(days=times[i])  
    data = values[i]
    mean = np.mean(data)    
    df.at[calendar.month_abbr[dt_time.month], dt_time.year] = mean - 273.15


df1 = df[[2005, 2010, 2015, 2020, 2025]]
ax = df1.plot.bar(rot=0, figsize=(12, 6))
ax.set_ylim(np.amin(np.amin(df1)), np.amax(np.amax(df1)))

plt.title(nc.variables['tas'].long_name)
plt.ylabel('°C')
plt.legend()
plt.show()

## 4 - Satellite observations

### 4.1 - Create a data folder

In [ ]:
#Get the destination folder path
folder = '/home/jovyan/work/c3s_data/c3s_NetCDF_examples/03_Satellite_observations'

import os 

#Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

zip_file = folder + '/sea_ice_concentration'

### 4.2 - Download a file
#### TO DO : Check you agree with the __[Terms of Use](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-sea-ice?tab=form)__ (Last section)
#### Note : Please, go to https://cds.climate.copernicus.eu/cdsapp#!/yourrequests to track the progress of your request

In [ ]:
import cdsapi

c = cdsapi.Client()

years = [
    '1998', '1999','2000',
    '2001','2002','2003',
    '2004','2005','2006',
    '2007','2008','2009',
    '2010','2012','2013', 
    '2014','2015','2016',
    '2017','2018'
]

c.retrieve(
    'satellite-sea-ice',
    {
        'format':'zip',
        'year': years,
        'month':['03'
        ],
        'variable':'sea_ice_concentration',
        'region':'north_hemisphere',
        'day':'01'
    },
    zip_file)

### 4.3 - Unzip the file

In [ ]:
import zipfile

zip = zipfile.ZipFile(zip_file)
zip.extractall(folder)

### 4.4 - Display a map

#### Note: The files must be in NetCDF format

In [ ]:
import os
os.environ["PROJ_LIB"] = "/opt/conda/share/proj";

from IPython.display import Image
import pygrib 
import numpy as np
import ipywidgets as widgets

from matplotlib.pyplot import figure
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import imageio
import netCDF4
from netCDF4 import Dataset
from datetime import datetime, timedelta
import glob

start_time = datetime.strptime('1978-01-01 00:00:00', '%Y-%m-%d  %H:%M:%S')
dt_times = []

files = glob.glob( folder + "/*.nc*")
files.sort()
images = []

i=0
for f in files:
    # Open a NetCDF file
    nc = Dataset(f, "r")

    param = 'ice_conc' 

    lats = nc.variables['lat'][:]  
    lons = nc.variables['lon'][:]
    times = nc.variables['time'][:]
    data = nc.variables[param][:]  

    unit = nc.variables[param].units
    param_name = nc.variables[param].long_name

    dt_times.append(start_time + timedelta(seconds=times[0]) )

    vmin = np.amin(data[0])
    vmax = np.amax(data[0])

    # Create and save a figure for each step
    figure(num=None, figsize=(16, 10))

    #Create a map
    m = Basemap(width=10000000,height=8000000,
                projection='laea',lat_ts=50,lat_0=90,lon_0=0.0)

    m.drawcoastlines()
    m.drawcountries()

    x,y = m(lons, lats)

    C = data[0]

    #Colour the map 
    m.pcolormesh(x, y, C, vmin=vmin, vmax = vmax, cmap=plt.cm.Blues_r)
    m.colorbar(label = unit) 

    plt.title(param_name + ' date: ' + dt_times[i].strftime('%Y-%m-%d  %H:%M:%S')) 
    #Save the figure
    plt.savefig(os.path.join(folder, 'sea_ice_' + str(i) + '.png'))
    plt.close()
    
    images.append(imageio.imread(os.path.join(folder, 'sea_ice_' + str(i) + '.png')))
                  
    i+=1

result_filename = "sea_ice_result.gif"

#Create a GIF
imageio.mimsave(os.path.join(folder, result_filename), images, 'GIF', duration=0.3)

display(Image(os.path.join(folder, result_filename)))

### 4.5 - Create a chart

In [ ]:
fig = figure(num=None, figsize=(14, 8))

# ys = [int(i) for i in years]
ax = fig.add_subplot(1, 1, 1)

#For each year, plot the data
means = []
for year in years:
    nc = Dataset(glob.glob( folder + "/*" + year + "*.nc*")[0], "r")

    unit = nc.variables['ice_conc'].units
    param_name = nc.variables['ice_conc'].long_name

    means.append(np.mean(nc.variables['ice_conc'][:][0]))


ax.set_ylim(np.amin(means), np.amax(means))

plt.bar(range(len(means)), means, alpha=0.5)

plt.title(param_name)
plt.xticks(range(len(means)), years)
plt.ylabel(unit)

plt.show()

## 5 - Sectoral climate indices

### 5.1 - Create a data folder

In [ ]:
#Get the destination folder path
folder = '/home/jovyan/work/c3s_data/c3s_NetCDF_examples/05_Sectoral climate indicess'

import os 

#Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

filename = folder + '/water_quantity' #Give a name to your file

### 5.3 - Download a file
#### TO DO : Check you agree with the  __[Terms of Use](https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-water-quantity-swicca?tab=form)__ (Last section)
#### Note : Please, go to https://cds.climate.copernicus.eu/cdsapp#!/yourrequests to track the progress of your request

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'sis-water-quantity-swicca',
    {
        'format':'zip',
        'variable':'wetness_1',
        'emissions_scenario':'rcp_8_5',
        'time_aggregation':'30_year_average',
        'period':'2011_2040',
        'horizontal_resolution':'0_5'
    },
    filename)

### 5.4 - Display a map

In [ ]:
import netCDF4
from netCDF4 import Dataset
nc =  Dataset(filename, 'r')
import matplotlib.colors as colors
import numpy.ma as ma

# Open a NetCDF file
nc = Dataset(filename, "r")

lons = nc.variables['lon'][:]  
lats = nc.variables['lat'][:]  
times = nc.variables['time'][:]

time = datetime.strptime('1951-01-01 00:00:00', '%Y-%m-%d  %H:%M:%S') \
    + timedelta(days =times[0])

unit = nc.variables['value1'].units

values = ma.array([nc.variables['value'  + str(k) ][:][0]  for k in range(1,13)])

figure(num=None, figsize=(16, 10))
m = Basemap(llcrnrlon=lons.min(), urcrnrlon=lons.max(),llcrnrlat=lats.min(),urcrnrlat=lats.max())

m.drawcoastlines()
m.drawcountries()

m.pcolormesh(lons, lats, values.mean(axis=0), cmap=plt.cm.jet) 

m.colorbar(label = unit)

plt.title('Wetness 1 ' + time.strftime('%Y-%m-%d  %H:%M:%S'))
plt.show()
